In [ ]:
%matplotlib inline

import copy
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import time

from numpy import diag, exp, log, sqrt, trace
from numpy.linalg import eig, inv, norm, solve, svd
from scipy.linalg import expm, solve_lyapunov, solve_sylvester, sqrtm

np.set_printoptions(precision=2, linewidth=1000)

$$d X_t = AX_t dt + B u_t dt + B d W_t$$

In [ ]:
N = 75 # 39
d = 2 * N
beta_1 = 1
beta_m = 1000
beta_N = 1
gamma_1 = 10
gamma_2 = 0.25 # 0.5
m = 1
xi = np.ones(N) # np.linspace(10, 10, N) * .1
eta = np.ones(N) # np.linspace(10, 10, N) * .1

Gamma = diag([gamma_1] + [gamma_2] * (N - 2) + [gamma_1])
M = diag([m] * N)
B = sqrt(2 * gamma_1 * m) * diag([0] * N + [1 / sqrt(beta_1)] + [0] * (N - 2) + [1 / sqrt(beta_N)]) # K
sigma = B[N:2*N, N:2*N]  
B_N = diag(-xi[:-1], 1) + diag(-xi[:-1], -1) + diag(xi + eta) + diag([0] + list(xi[1:-1]) + [0])
A = np.zeros([2 * N, 2 * N])
A[N:2*N, N:2*N] = -Gamma
A[N:2*N, 0:N] = -B_N
A[0:N, N:2*N] = inv(M)

C = np.eye(2 * N)

#C = np.zeros([4, d])
#C[0, 0] = 1
#C[1, N - 1] = 1
#C[2, N] = 1
#C[3, 2 * N - 1] = 1

if ~np.all(np.linalg.eigvals(A) < 0):
    print('not all EV of A are negative')
    
Sigma = np.zeros([2 * N, 2 * N])
Sigma[0:N, 0:N] = inv(B_N)
Sigma[N:2*N, N:2*N] = M

sigma_inf = solve_lyapunov(A, -B.dot(B.T))
lambda_min = np.min(np.real(np.linalg.eigvals(-A)))
exp(-2 * lambda_min * 100) # see Theorem 3.22 in Lara's thesis 

P = solve_lyapunov(A, -B.dot(B.T)) # + np.ones(N).dot(np.ones(N).T)
Q = solve_lyapunov(A.T, -C.T.dot(C))

#K = sqrtm(P)
#L = sqrtm(Q)

V_P, sigmas_P, U_P_T = svd(P)
K = V_P.dot(sqrt(diag(sigmas_P))).dot(U_P_T)

V_Q, sigmas_Q, U_Q_T = svd(Q)
L = V_Q.dot(sqrt(diag(sigmas_Q))).dot(U_Q_T)

V, sigmas, U_T = svd(K.T.dot(L))
Hankel = np.diag(sigmas)
Sigma_inv_05 = np.diag(1 / sqrt(sigmas))

T = Sigma_inv_05.dot(U_T).dot(L.T)
T_inv = K.dot(V).dot(Sigma_inv_05)
A_ = np.real(T.dot(A).dot(T_inv))
B_ = np.real(T.dot(B))
C_ = np.real(C.dot(T_inv))

fig, ax = plt.subplots(1, 2, figsize=(15, 4))
plt.suptitle(r'Hankel singular values, $m=%.2f, \beta=%.2f, \gamma_1=%.2f, \xi=%.2f, \eta=%.2f$' % (m, beta_1, gamma_1, xi[0], eta[0]))
ax[0].scatter(range(1, 2 * N + 1), sigmas, s=2);
ax[0].set_yscale('log');
ax[0].set_ylim(0.5 * min(sigmas), 2 * max(sigmas));
ax[0].set_title(r'$\sigma_i$')
ax[1].plot(range(1, 2 * N + 1), [sum(sigmas[i:]) for i in range(2 * N)])
ax[1].set_yscale('log');
ax[1].set_title(r'$\sum_{k = i}^{2N}\sigma_k$');

H = np.zeros([d, d * d])
for i in range(d):
    H[:, i * d:i * d + d] = np.linalg.matrix_power(A, i).dot(B)
    
np.linalg.matrix_rank(H)

T_end = 100
lambda_min = np.min(np.real(np.linalg.eigvals(-A)))
np.exp(-2 * lambda_min * T_end) # see Theorem 3.22 in Lara's thesis 

In [ ]:
## sanity check

B1 = np.zeros([d, d])
B_in = np.zeros([d, d])

A_hat = np.zeros([d + d, d + d])
A_hat[0:d, 0:d] = A
A_hat[d:d + d, d:d + d] = A
B1_hat = np.concatenate([B1, B1])
B2_hat = np.concatenate([B, B])
B_in_hat = np.concatenate([B_in, B_in])
C_hat = np.concatenate([C, -C], axis=1)

R_hat = B_in_hat.dot(B_in_hat.T)
#P = solve_lyapunov(A, -B.dot(B.T) + A.dot(R) + R.dot(A.T))# + 
P_hat = solve_lyapunov(A_hat, -B1_hat.dot(B1_hat.T) + A_hat.dot(R_hat + B2_hat.dot(B2_hat.T))
                         + (R_hat + B2_hat.dot(B2_hat.T)).dot(A_hat.T))
Q_hat = solve_lyapunov(A_hat.T, -C_hat.T.dot(C_hat))

#K_hat = sqrtm(P_hat)
#L_hat = sqrtm(Q_hat)

V_P_hat, sigmas_P_hat, U_P_hat_T = svd(P_hat)
K_hat = V_P_hat.dot(sqrt(diag(sigmas_P_hat))).dot(U_P_hat_T)

V_Q_hat, sigmas_Q_hat, U_Q_hat_T = svd(Q_hat)
L_hat = V_Q_hat.dot(sqrt(diag(sigmas_Q_hat))).dot(U_Q_hat_T)

V_hat, sigmas_hat, U_hat_T = svd(K_hat.T.dot(L_hat))
Sigma_hat = np.diag(sigmas_hat)
Sigma_hat_inv_05 = np.diag(1 / sqrt(sigmas_hat))

sum(sigmas_hat)

In [ ]:
def apply_model_reduction(r, A_, B_, C_, mode='BT'):
    A_11 = A_[:r, :r]
    A_12 = A_[:r, r:]
    A_21 = A_[r:, :r]
    A_22 = A_[r:, r:]
    B_1 = B_[:r, :]
    B_2 = B_[r:, :]
    C_1 = C_[:, :r]
    C_2 = C_[:, r:]
    
    if mode == 'BT':
        # Balanced truncation
        A_r = A_11
        B_r = B_1
        C_r = C_1
        
    elif mode == 'SPA':
        # Singular perturbation approximation
        A_r = A_11 - A_12.dot(inv(A_22)).dot(A_21)
        B_r = B_1
        C_r = C_1 - C_2.dot(inv(A_22)).dot(A_21)
    return A_r, B_r, C_r

In [ ]:
u_L2 = 0

general_bound = []
L2_bound = []
L2_bound_ = []
simulation_max_sqrt = []
simulation_max = []
simulation_sqrt_L2 = []

T_end = 10
delta_t = 0.01
N_ = int(np.ceil(T_end / delta_t))
K_ = 1000

r_range = range(1, d, 1)

for r in r_range:

    A_r, B_r, C_r = apply_model_reduction(r, A_, B_, C_, mode='BT')
    
    B1 = np.zeros([d, d])
    B1_r = np.zeros([r, d])
    B_in = np.zeros([d, d])

    P_r = solve_lyapunov(A_r, -B_r.dot(B_r.T))
    P_g = solve_sylvester(A, A_r.T, -B.dot(B_r.T))
    Sigma_2 = np.diag(sigmas[r:])
    P_g1 = T.dot(P_g)[:r, :]
    P_g2 = T.dot(P_g)[r:, :]
    
    A_12 = A_[:r, r:]
    A_21 = A_[r:, :r]
    A_22 = A_[r:, r:]
    B_2 = B_[r:, :]
    
    #B1_r = B_r[:, :d]
    #B2_r = B_r[:, d:]
    
    A_hat = np.zeros([d + r, d + r])
    A_hat[0:d, 0:d] = A
    A_hat[d:d + r, d:d + r] = A_r
    B1_hat = np.concatenate([B1, B1_r])
    B2_hat = np.concatenate([B, B_r])
    B_in_hat = np.concatenate([B_in, T.dot(B_in)[:r, :]])
    C_hat = np.concatenate([C, -C_r], axis=1)

    R_hat = B_in_hat.dot(B_in_hat.T)
    P_hat = solve_lyapunov(A_hat, -B1_hat.dot(B1_hat.T) + A_hat.dot(R_hat + B2_hat.dot(B2_hat.T))
                             + (R_hat + B2_hat.dot(B2_hat.T)).dot(A_hat.T))
    Q_hat = solve_lyapunov(A_hat.T, -C_hat.T.dot(C_hat))

    #K_hat = sqrtm(P_hat)
    #L_hat = sqrtm(Q_hat)
    
    V_P_hat, sigmas_P_hat, U_P_hat_T = svd(P_hat)
    K_hat = V_P_hat.dot(sqrt(diag(sigmas_P_hat))).dot(U_P_hat_T)

    V_Q_hat, sigmas_Q_hat, U_Q_hat_T = svd(Q_hat)
    L_hat = V_Q_hat.dot(sqrt(diag(sigmas_Q_hat))).dot(U_Q_hat_T)

    V_hat, sigmas_hat, U_hat_T = svd(K_hat.T.dot(L_hat))
    Sigma_hat = np.diag(sigmas_hat)
    Sigma_hat_inv_05 = np.diag(1 / sqrt(sigmas_hat))

    # bounds
    general_bound.append(sqrt(2) * max(1, u_L2) * sqrt(np.trace(C.dot(P).dot(C.T) + C_r.dot(P_r).dot(C_r.T) - 2 * C.dot(P_g).dot(C_r.T))))
    L2_bound.append(sum(sigmas[r:]) * (sqrt(T_end) + 2 * u_L2))    
    L2_bound_.append(sum(sigmas_hat) * (sqrt(T_end) + 2 * u_L2))

    #X = np.zeros([K_, d, N_ + 1])
    #xi = np.zeros([K_, d])
    #
    #for i in range(N_):
    #    xi[:, N] = sqrt(2 * gamma / beta_1) * np.random.randn(K_)
    #    xi[:, 2 * N - 1] = sqrt(2 * m * gamma / beta_N) * np.random.randn(K_)
    #    X[:, :, i + 1] = X[:, :, i] + A.dot(X[:, :, i].T).T * delta_t + B.dot(u(X[:, :, i], i * delta_t).T).T * delta_t + xi * sqrt(delta_t)

    X = np.zeros([K_, d])
    X_r = np.zeros([K_, r])
    xi = np.zeros([K_, d])

    diff_max_sqrt = []
    diff_sqrt_L2 = np.zeros(K_)

    for i in range(N_):
        xi = np.random.randn(K_, d)
        #xi[:, N] = sqrt(2 * gamma / beta_1) * np.random.randn(K_)
        #xi[:, 2 * N - 1] = sqrt(2 * m * gamma / beta_N) * np.random.randn(K_)
        X = X + A.dot(X.T).T * delta_t + B.dot(xi.T).T * sqrt(delta_t) # xi * sqrt(delta_t)
        X_r = X_r + A_r.dot(X_r.T).T * delta_t + B_r.dot(xi.T).T * sqrt(delta_t)

        Y = C.dot(X.T).T
        Y_r = C_r.dot(X_r.T).T
        diff_max_sqrt.append(sqrt(np.mean(np.sum((Y - Y_r)**2, 1))))
        diff_sqrt_L2 += np.sum((Y - Y_r)**2, 1) * delta_t
    
    simulation_max_sqrt.append(np.max(diff_max_sqrt))
    #simulation_max.append(np.max(np.mean(sqrt(np.sum((Y - Y_r)**2, 2)), 0)))
    simulation_sqrt_L2.append(np.mean(sqrt(diff_sqrt_L2)))

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle(r'Ornstein-Uhlenbeck, $d = %d, T = %.0f, u = 0$' % (d, T_end))
ax[0].set_title('supremum error')
ax[0].scatter(r_range, np.real(np.array(general_bound)), marker='.', label='bound Theorem 1')
ax[0].scatter(r_range, np.array(simulation_max_sqrt), marker='.', label=r'simulation Theorem 1') # $\max_t \sqrt{\mathbb{E}[ \|Y_t - Y_t^r\|^2]}$
#ax[0].scatter(r_range, np.array(simulation_max), marker='.', label=r'simulation, $\max_t \mathbb{E}[ \|Y_t - Y_t^r\|]$')
ax[0].legend()
ax[0].set_yscale('log')
ax[0].set_ylim(0.8 * min(np.real(np.array(simulation_max_sqrt))), 1.2 * max(np.real(np.array(general_bound))))
ax[1].set_title(r'$L^2$ error')
ax[1].scatter(r_range, np.array(L2_bound_), marker='.', label=r'bound Theorem 2')
ax[1].scatter(r_range, np.array(simulation_sqrt_L2), marker='.', label=r'simulation Theorem 2') # $\mathbb{E}[\|Y_t - Y_t^r\|_{L_2}]$
#ax[1].scatter(r_range, np.array(L2_bound), marker='.', label='Hankel sum')
ax[1].legend()
ax[1].set_yscale('log')
ax[1].set_ylim(0.8 * min(simulation_sqrt_L2), 1.2 * max(L2_bound))
ax[2].set_title('Hankel singular values')
ax[2].scatter(range(d), sigmas, marker='.');
ax[2].set_yscale('log');
ax[2].set_ylim(0.8 * min(sigmas), 1.2 * max(sigmas));

fig.tight_layout(rect=[0, 0.03, 1, 0.95])

fig.savefig('img/chain_of_oscillators_d150.pdf')

In [ ]:
#temp = np.eye(2 * N) + np.abs(np.random.randn(2 * N, 2 * N)) * 0.1
#Sigma = temp + temp.T
#Sigma[N:2*N, 0:N] = 0
#Sigma[0:N, N:2*N] = 0

#Sigma = np.zeros([2 * N, 2 * N])
#Sigma[0:N, 0:N] = inv(B_N)
#Sigma[N:2*N, N:2*N] = M

#Sigma = Sigma[2 * selection, :]
#Sigma = Sigma[:, 2 * selection]

Pi_ = np.linalg.pinv(np.kron(Sigma, B.dot(B.T)) + np.kron(B.dot(B.T), Sigma)).dot(
        np.ndarray.flatten(A.dot(Sigma) + Sigma.dot(A.T) + B.dot(B.T), order='F')).reshape(2 * N, 2 * N, order='F')

#sigmasigmaT_inv = 1 / (2 * m * gamma) * diag([beta_1] + [0] * (N - 2) + [beta_N]) # K
#sigmasigmaT_inv = inv(sigma.dot(sigma.T))
#Pi = np.zeros([2 * N, 2 * N])
#Pi[N:2*N, N:2*N] = 0.5 * inv(M) - sigmasigmaT_inv.dot(Gamma)

if ~np.all(np.real(np.linalg.eigvals(A - B.dot(B.T).dot(Pi_))) < 0):
    print('not all EV of A are negative')

In [ ]:
# check convergence speed 
#T_end = 100
#lambda_min = np.min(np.real(np.linalg.eigvals(-A + B.dot(B.T).dot(Pi_))))
#np.exp(-2 * lambda_min * T_end)

In [ ]:
H = np.zeros([2 * d, 2 * d])
H[:d, :d] = A.dot(Sigma) + Sigma.dot(A.T) + B.dot(B.T)
H[:d, d:2 * d] = B
H[d:2 * d, :d] = B
print(np.linalg.matrix_rank(H))

H = np.zeros([2 * d, 2 * d])
H[:d, d:2 * d] = B
H[d:2 * d, :d] = B
print(np.linalg.matrix_rank(H))

In [ ]:
np.sum(np.abs((A - B.dot(B.T).dot(Pi_)).dot(Sigma) + Sigma.dot((A - B.dot(B.T).dot(Pi_)).T) + B.dot(B)))

In [ ]:
#np.sum(np.abs((A - B.dot(B.T).dot(Pi)).dot(Sigma) + Sigma.dot((A - B.dot(B.T).dot(Pi)).T) + B.dot(B)))

In [ ]:
#sigmasigmaT_inv = 1 / (2 * m * gamma) * diag([beta_1] + [0] * (N - 2) + [beta_N]) # K
#Pi = np.zeros([2 * N, 2 * N])
#Pi[N:2*N, N:2*N] = 0.5 * inv(M) - sigmasigmaT_inv.dot(Gamma)

def u(x, t):
    #return np.zeros([d, d]).dot(x.T).T
    return -B.T.dot(Pi_).dot(x.T).T

d = 2 * N

T_end = 100
delta_t = 0.001
N_ = int(np.ceil(T_end / delta_t))
K_ = 1000

X = np.zeros([K_, d])
xi = np.zeros([K_, d])

for i in range(N_):
    #xi[:, N] = sqrt(2 * gamma / beta_1) * np.random.randn(K_)
    #xi[:, 2 * N - 1] = sqrt(2 * m * gamma / beta_N) * np.random.randn(K_)
    xi = np.random.randn(K_, d)
    
    X = X + A.dot(X.T).T * delta_t + B.dot(u(X, i * delta_t).T).T * delta_t + B.dot(xi.T).T * sqrt(delta_t)

In [ ]:
np.set_printoptions(precision=1, linewidth=1000)

In [ ]:
sqrt(np.sum((Sigma - np.cov(X.T))**2)) / d

In [ ]:
Sigma[np.arange(d), np.arange(d)]

In [ ]:
np.cov(X.T)[np.arange(d), np.arange(d)]

## optimal steering for reduced system

In [ ]:
Sigma_Y = np.eye(4)

r = 4

temp = 3 * np.eye(r) + np.abs(np.random.randn(r, r)) * 1
Sigma_Y = temp + temp.T

A_r, B_r, C_r = apply_model_reduction(r, A_, B_, C_, mode='BT')

Sigma_r = np.linalg.pinv(C_r).dot(Sigma_Y).dot(np.linalg.pinv(C_r.T))

H = np.zeros([r + d, r + d])
H[:r, :r] = A_r.dot(Sigma_r) + Sigma_r.dot(A_r.T) + B_r.dot(B_r.T)
H[:r, r:r + d] = B_r
H[r:r + d, :r] = B_r.T
print(np.linalg.matrix_rank(H))

H = np.zeros([r + d, r + d])
H[:r, r:r + d] = B_r
H[r:r + d, :r] = B_r.T
print(np.linalg.matrix_rank(H))

#Sigma = np.zeros([2 * N, 2 * N])
#Sigma[0:N, 0:N] = inv(B_N)
#Sigma[N:2*N, N:2*N] = M

Pi_r = np.linalg.pinv(np.kron(Sigma_r, B_r.dot(B_r.T)) + np.kron(B_r.dot(B_r.T), Sigma_r)).dot(
        np.ndarray.flatten(A_r.dot(Sigma_r) + Sigma_r.dot(A_r.T) + B_r.dot(B_r.T), order='F')).reshape(r, r, order='F')

if ~np.all(np.real(np.linalg.eigvals(A_r - B_r.dot(B_r.T).dot(Pi_r))) < 0):
    print('not all EV of A are negative')

In [ ]:
np.sum(np.abs((A_r - B_r.dot(B_r.T).dot(Pi_r)).dot(Sigma_r) + Sigma_r.dot((A_r - B_r.dot(B_r.T).dot(Pi_r)).T) + B_r.dot(B_r.T)))

In [ ]:
lambda_min = np.min(np.real(np.linalg.eigvals(-A_r + B_r.dot(B_r.T).dot(Pi_r))))
exp(-2 * lambda_min * 100) # see Theorem 3.22 in Lara's thesis 

In [ ]:
Sigma_Y_norms = []
K_delta_t = [[100, 0.01], [1000, 0.01], [10000, 0.01], [10000, 0.001]]

def u(x, t):
    #return np.zeros([d, d]).dot(x.T).T
    return -B.T.dot(Pi_).dot(x.T).T

def u_r(x, t):
    #return np.zeros([d, d]).dot(x.T).T
    return -B_r.T.dot(Pi_r).dot(x.T).T

#sigmasigmaT_inv = 1 / (2 * m * gamma) * diag([beta_1] + [0] * (N - 2) + [beta_N]) # K
#Pi = np.zeros([2 * N, 2 * N])
#Pi[N:2*N, N:2*N] = 0.5 * inv(M) - sigmasigmaT_inv.dot(Gamma)

for j, [K_, delta_t] in enumerate(K_delta_t):
    
    Sigma_Y_norms.append([])

    d = 2 * N

    T_end = 30
    N_ = int(np.ceil(T_end / delta_t))

    X = np.zeros([K_, d])
    X_r = np.zeros([K_, r])
    xi = np.zeros([K_, d])

    for i in range(N_):
        #xi[:, N] = sqrt(2 * gamma / beta_1) * np.random.randn(K_)
        #xi[:, 2 * N - 1] = sqrt(2 * m * gamma / beta_N) * np.random.randn(K_)
        xi = np.random.randn(K_, d)

        #X = X + A.dot(X.T).T * delta_t + B.dot(u(X, i * delta_t).T).T * delta_t + B.dot(xi.T).T * sqrt(delta_t)
        X_r = X_r + A_r.dot(X_r.T).T * delta_t + B_r.dot(u_r(X_r, i * delta_t).T).T * delta_t + B_r.dot(xi.T).T * sqrt(delta_t)

        if i % 1 == 0:
            Y_r = C_r.dot(X_r.T).T
            Sigma_Y_norms[j].append(sqrt(np.sum((Sigma_Y - np.cov(Y_r.T))**2)) / d)

    #Y = C.dot(X.T).T
    Y_r = C_r.dot(X_r.T).T

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

for j, [K_, delta_t] in enumerate(K_delta_t):
    ax.plot(np.linspace(0, T_end, len(Sigma_Y_norms[j])), Sigma_Y_norms[j], label=r'$k = %d, \Delta t = %.3f$' % (K_, delta_t));
#ax.set_yscale('log')
ax.legend();
ax.set_xlabel('t');
ax.set_ylabel(r'$\frac{1}{d}\|\Sigma - \hat{\Sigma}_t\|_F$');
ax.set_xlim(-0.5, 15);
#fig.savefig('img/chain_of_oscillators_reduced_stationary.pdf')